In [ ]:
import os
import time
import math
import random
import numpy as np
import pandas as pd
from google import genai
from google.genai import types
from dotenv import load_dotenv

In [ ]:
random.seed(42)
load_dotenv()
client = genai.Client(api_key=os.getenv("GEMINI_API_KEY"))
prompt_template = ''

In [16]:
def build_prompt(text):
    return prompt_template.replace('KALIMAT:', text)

In [17]:
def load_data():
    data = pd.read_csv('./val_data.csv', encoding='latin-1')
    return data

In [18]:
safety_settings = [
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
    types.SafetySetting(
        category=types.HarmCategory.HARM_CATEGORY_CIVIC_INTEGRITY,
        threshold=types.HarmBlockThreshold.BLOCK_NONE,
    ),
]

def generate_noise(prompt):
    response = client.models.generate_content(
        model="gemini-2.5-flash-preview-04-17", 
        contents=prompt, 
        config=types.GenerateContentConfig(
            safety_settings=safety_settings,
        )
    )
    return response.text

# CHAR LEVEL NOISE

In [22]:
# Typos
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mensimulasikan typo pada kalimat tersebut.
- lakukan pada maksimal 25% dari kata yang ada pada kalimat tersebut.
- output hanya berupa kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil'''

In [23]:
data = load_data()
data.head()

,No,answer,1-FR,2-GI,3-PI,4-DM,5-EDTRB,6-RE,Text_Clean,filtered_text,token,tokens_stemmed,Process_Data
0,401,"Dear Ressa, Sakit bagian dada dapat disebabkan...",1,0,1,1,1,0,dear ressa sakit bagian dada dapat disebabkan ...,dear ressa sakit dada disebabkan cedera otot k...,"['dear', 'ressa', 'sakit', 'dada', 'disebabkan...","['dear', 'ressa', 'sakit', 'dada', 'sebab', 'c...",dear ressa sakit dada sebab cedera otot kram o...
1,402,"Alo, Terima kasih atas pertanyaannya. Perlu di...",1,0,1,1,1,0,alo terima kasih atas pertanyaannya perlu dipa...,alo terima kasih pertanyaannya alami pembengka...,"['alo', 'terima', 'kasih', 'pertanyaannya', 'a...","['alo', 'terima', 'kasih', 'tanya', 'alami', '...",alo terima kasih tanya alami bengkak benjol le...
2,403,Halo Virus HIV dapat ditularkan dengan adanya ...,1,0,1,1,1,0,halo virus hiv dapat ditularkan dengan adanya ...,halo virus hiv ditularkan kondisi bersamaan pe...,"['halo', 'virus', 'hiv', 'ditularkan', 'kondis...","['halo', 'virus', 'hiv', 'tular', 'kondisi', '...",halo virus hiv tular kondisi sama pindah cair ...
3,404,Hallo Imam Rasa gatal bisa timbul akibat ban...,1,0,1,1,1,0,hallo imam rasa gatal bisa timbul akibat banya...,hallo imam gatal timbul akibat faktor reaksi a...,"['hallo', 'imam', 'gatal', 'timbul', 'akibat',...","['hallo', 'imam', 'gatal', 'timbul', 'akibat',...",hallo imam gatal timbul akibat faktor reaksi a...
4,405,"Halo, Profertil berisi Clomiphene Citrate ya...",1,0,1,0,1,0,halo profertil berisi clomiphene citrate yang ...,halo profertil berisi clomiphene citrate obat ...,"['halo', 'profertil', 'berisi', 'clomiphene', ...","['halo', 'profertil', 'isi', 'clomiphene', 'ci...",halo profertil isi clomiphene citrate obat ran...


In [24]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Text_Clean']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["Text_Clean"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(2)

Index: 1
Original: alo terima kasih atas pertanyaannya perlu dipastikan kembali apakah yang anda alami berupa pembengkakan atau benjolan di leher karena kedua keluhan ini bisa disebabkan oleh kondisi yang berbeda beberapa kondisi medis dapat menyebabkan gejala yang anda alami sehingga untuk memastikannya diperlukan pemeriksaan dokter lebih lanjut beberapa penyebabnya misalnya reaksi alergi infeksi virus bakteri limfoma penyakit autoimun apabila gejala ini sudah berlangsung selama beberapa waktu silakan anda memeriksakan diri ke dokter umum terdekat dulu dokter perlu bertanya lebih lanjut mengenai gejala anda memeriksa anda dan bila perlu melakukan pemeriksaan lain seperti pemeriksaan darah untuk memastikan sebabnya dari pemeriksaan ini dokter akan memberikan penanganan lebih lanjut termasuk merujuk anda ke dokter spesialis bila diperlukan untuk sementara silakan melakukan anjuran berikut untuk mengurangi gejala anda mengkonsumsi parasetamol sesuai dengan aturan pakai di kemasan obat un

In [25]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 13


In [26]:
data.to_csv('./noisy/typo.csv', index=False)

# SYNONYM REPLACEMENT

In [ ]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan mengganti maksimum 25% kata dalam kalimat dengan sinonim nya
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [ ]:
data = load_data()
data.head()

,No,answer,1-FR,2-GI,3-PI,4-DM,5-EDTRB,6-RE,Text_Clean,filtered_text,token,tokens_stemmed,Process_Data
0,401,"Dear Ressa, Sakit bagian dada dapat disebabkan...",1,0,1,1,1,0,dear ressa sakit bagian dada dapat disebabkan ...,dear ressa sakit dada disebabkan cedera otot k...,"['dear', 'ressa', 'sakit', 'dada', 'disebabkan...","['dear', 'ressa', 'sakit', 'dada', 'sebab', 'c...",dear ressa sakit dada sebab cedera otot kram o...
1,402,"Alo, Terima kasih atas pertanyaannya. Perlu di...",1,0,1,1,1,0,alo terima kasih atas pertanyaannya perlu dipa...,alo terima kasih pertanyaannya alami pembengka...,"['alo', 'terima', 'kasih', 'pertanyaannya', 'a...","['alo', 'terima', 'kasih', 'tanya', 'alami', '...",alo terima kasih tanya alami bengkak benjol le...
2,403,Halo Virus HIV dapat ditularkan dengan adanya ...,1,0,1,1,1,0,halo virus hiv dapat ditularkan dengan adanya ...,halo virus hiv ditularkan kondisi bersamaan pe...,"['halo', 'virus', 'hiv', 'ditularkan', 'kondis...","['halo', 'virus', 'hiv', 'tular', 'kondisi', '...",halo virus hiv tular kondisi sama pindah cair ...
3,404,Hallo Imam Rasa gatal bisa timbul akibat ban...,1,0,1,1,1,0,hallo imam rasa gatal bisa timbul akibat banya...,hallo imam gatal timbul akibat faktor reaksi a...,"['hallo', 'imam', 'gatal', 'timbul', 'akibat',...","['hallo', 'imam', 'gatal', 'timbul', 'akibat',...",hallo imam gatal timbul akibat faktor reaksi a...
4,405,"Halo, Profertil berisi Clomiphene Citrate ya...",1,0,1,0,1,0,halo profertil berisi clomiphene citrate yang ...,halo profertil berisi clomiphene citrate obat ...,"['halo', 'profertil', 'berisi', 'clomiphene', ...","['halo', 'profertil', 'isi', 'clomiphene', 'ci...",halo profertil isi clomiphene citrate obat ran...


In [ ]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Text_Clean']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["Text_Clean"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        time.sleep(2)

Index: 1
Original: alo terima kasih atas pertanyaannya perlu dipastikan kembali apakah yang anda alami berupa pembengkakan atau benjolan di leher karena kedua keluhan ini bisa disebabkan oleh kondisi yang berbeda beberapa kondisi medis dapat menyebabkan gejala yang anda alami sehingga untuk memastikannya diperlukan pemeriksaan dokter lebih lanjut beberapa penyebabnya misalnya reaksi alergi infeksi virus bakteri limfoma penyakit autoimun apabila gejala ini sudah berlangsung selama beberapa waktu silakan anda memeriksakan diri ke dokter umum terdekat dulu dokter perlu bertanya lebih lanjut mengenai gejala anda memeriksa anda dan bila perlu melakukan pemeriksaan lain seperti pemeriksaan darah untuk memastikan sebabnya dari pemeriksaan ini dokter akan memberikan penanganan lebih lanjut termasuk merujuk anda ke dokter spesialis bila diperlukan untuk sementara silakan melakukan anjuran berikut untuk mengurangi gejala anda mengkonsumsi parasetamol sesuai dengan aturan pakai di kemasan obat un

In [ ]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 13


In [16]:
data.to_csv('./noisy/synonym-replacement.csv', index=False)

# WORD INSERTION

In [29]:
prompt_template = f'''KALIMAT:

- lakukan noise injection dengan menyisipkan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- sebar 15-30 kata tambahan di seluruh kalimat
- jangan mengubah kata yang sudah ada, hanya sisipkan kata baru
- output hanya berupa 1 baris kalimat akhir yang telah diubah
- jangan tambahkan markdown dan tanda baca apapun
- jangan hilangkan kata selain dari kata yang diganti
- gunakan huruf kecil'''

In [30]:
data = load_data()
data.head()

,No,answer,1-FR,2-GI,3-PI,4-DM,5-EDTRB,6-RE,Text_Clean,filtered_text,token,tokens_stemmed,Process_Data
0,401,"Dear Ressa, Sakit bagian dada dapat disebabkan...",1,0,1,1,1,0,dear ressa sakit bagian dada dapat disebabkan ...,dear ressa sakit dada disebabkan cedera otot k...,"['dear', 'ressa', 'sakit', 'dada', 'disebabkan...","['dear', 'ressa', 'sakit', 'dada', 'sebab', 'c...",dear ressa sakit dada sebab cedera otot kram o...
1,402,"Alo, Terima kasih atas pertanyaannya. Perlu di...",1,0,1,1,1,0,alo terima kasih atas pertanyaannya perlu dipa...,alo terima kasih pertanyaannya alami pembengka...,"['alo', 'terima', 'kasih', 'pertanyaannya', 'a...","['alo', 'terima', 'kasih', 'tanya', 'alami', '...",alo terima kasih tanya alami bengkak benjol le...
2,403,Halo Virus HIV dapat ditularkan dengan adanya ...,1,0,1,1,1,0,halo virus hiv dapat ditularkan dengan adanya ...,halo virus hiv ditularkan kondisi bersamaan pe...,"['halo', 'virus', 'hiv', 'ditularkan', 'kondis...","['halo', 'virus', 'hiv', 'tular', 'kondisi', '...",halo virus hiv tular kondisi sama pindah cair ...
3,404,Hallo Imam Rasa gatal bisa timbul akibat ban...,1,0,1,1,1,0,hallo imam rasa gatal bisa timbul akibat banya...,hallo imam gatal timbul akibat faktor reaksi a...,"['hallo', 'imam', 'gatal', 'timbul', 'akibat',...","['hallo', 'imam', 'gatal', 'timbul', 'akibat',...",hallo imam gatal timbul akibat faktor reaksi a...
4,405,"Halo, Profertil berisi Clomiphene Citrate ya...",1,0,1,0,1,0,halo profertil berisi clomiphene citrate yang ...,halo profertil berisi clomiphene citrate obat ...,"['halo', 'profertil', 'berisi', 'clomiphene', ...","['halo', 'profertil', 'isi', 'clomiphene', 'ci...",halo profertil isi clomiphene citrate obat ran...


In [31]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Text_Clean']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["Text_Clean"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(4)

Index: 1
Original: alo terima kasih atas pertanyaannya perlu dipastikan kembali apakah yang anda alami berupa pembengkakan atau benjolan di leher karena kedua keluhan ini bisa disebabkan oleh kondisi yang berbeda beberapa kondisi medis dapat menyebabkan gejala yang anda alami sehingga untuk memastikannya diperlukan pemeriksaan dokter lebih lanjut beberapa penyebabnya misalnya reaksi alergi infeksi virus bakteri limfoma penyakit autoimun apabila gejala ini sudah berlangsung selama beberapa waktu silakan anda memeriksakan diri ke dokter umum terdekat dulu dokter perlu bertanya lebih lanjut mengenai gejala anda memeriksa anda dan bila perlu melakukan pemeriksaan lain seperti pemeriksaan darah untuk memastikan sebabnya dari pemeriksaan ini dokter akan memberikan penanganan lebih lanjut termasuk merujuk anda ke dokter spesialis bila diperlukan untuk sementara silakan melakukan anjuran berikut untuk mengurangi gejala anda mengkonsumsi parasetamol sesuai dengan aturan pakai di kemasan obat un

In [32]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 13


In [33]:
data.to_csv('./noisy/word-insertion.csv', index=False)

# WORD DELETION

In [6]:
data = load_data()
data.head()

,No,answer,1-FR,2-GI,3-PI,4-DM,5-EDTRB,6-RE,Text_Clean,filtered_text,token,tokens_stemmed,Process_Data
0,401,"Dear Ressa, Sakit bagian dada dapat disebabkan...",1,0,1,1,1,0,dear ressa sakit bagian dada dapat disebabkan ...,dear ressa sakit dada disebabkan cedera otot k...,"['dear', 'ressa', 'sakit', 'dada', 'disebabkan...","['dear', 'ressa', 'sakit', 'dada', 'sebab', 'c...",dear ressa sakit dada sebab cedera otot kram o...
1,402,"Alo, Terima kasih atas pertanyaannya. Perlu di...",1,0,1,1,1,0,alo terima kasih atas pertanyaannya perlu dipa...,alo terima kasih pertanyaannya alami pembengka...,"['alo', 'terima', 'kasih', 'pertanyaannya', 'a...","['alo', 'terima', 'kasih', 'tanya', 'alami', '...",alo terima kasih tanya alami bengkak benjol le...
2,403,Halo Virus HIV dapat ditularkan dengan adanya ...,1,0,1,1,1,0,halo virus hiv dapat ditularkan dengan adanya ...,halo virus hiv ditularkan kondisi bersamaan pe...,"['halo', 'virus', 'hiv', 'ditularkan', 'kondis...","['halo', 'virus', 'hiv', 'tular', 'kondisi', '...",halo virus hiv tular kondisi sama pindah cair ...
3,404,Hallo Imam Rasa gatal bisa timbul akibat ban...,1,0,1,1,1,0,hallo imam rasa gatal bisa timbul akibat banya...,hallo imam gatal timbul akibat faktor reaksi a...,"['hallo', 'imam', 'gatal', 'timbul', 'akibat',...","['hallo', 'imam', 'gatal', 'timbul', 'akibat',...",hallo imam gatal timbul akibat faktor reaksi a...
4,405,"Halo, Profertil berisi Clomiphene Citrate ya...",1,0,1,0,1,0,halo profertil berisi clomiphene citrate yang ...,halo profertil berisi clomiphene citrate obat ...,"['halo', 'profertil', 'berisi', 'clomiphene', ...","['halo', 'profertil', 'isi', 'clomiphene', 'ci...",halo profertil isi clomiphene citrate obat ran...


In [7]:
random.seed(42)
augmented_cnt = 0
for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Text_Clean']
        words = text.split()
        num_words = len(words)
        num_to_del = math.ceil(len(text.split(' ')) / 4)

        print("Index:", i)
        print("Original:", text, '\n')

        if num_to_del > 0:
            indices_to_delete = np.random.choice(num_words, size=num_to_del, replace=False)
            mask = np.ones(num_words, dtype=bool)
            mask[indices_to_delete] = False
            augmented_words = np.array(words)[mask]
            augmented = ' '.join(augmented_words)
        else:
            augmented = text  # if too short, don't change

        data.at[i, "Text_Clean"] = augmented

        print("Augmented:", augmented)
        print("=" * 50)
        # time.sleep(5)

Index: 1
Original: alo terima kasih atas pertanyaannya perlu dipastikan kembali apakah yang anda alami berupa pembengkakan atau benjolan di leher karena kedua keluhan ini bisa disebabkan oleh kondisi yang berbeda beberapa kondisi medis dapat menyebabkan gejala yang anda alami sehingga untuk memastikannya diperlukan pemeriksaan dokter lebih lanjut beberapa penyebabnya misalnya reaksi alergi infeksi virus bakteri limfoma penyakit autoimun apabila gejala ini sudah berlangsung selama beberapa waktu silakan anda memeriksakan diri ke dokter umum terdekat dulu dokter perlu bertanya lebih lanjut mengenai gejala anda memeriksa anda dan bila perlu melakukan pemeriksaan lain seperti pemeriksaan darah untuk memastikan sebabnya dari pemeriksaan ini dokter akan memberikan penanganan lebih lanjut termasuk merujuk anda ke dokter spesialis bila diperlukan untuk sementara silakan melakukan anjuran berikut untuk mengurangi gejala anda mengkonsumsi parasetamol sesuai dengan aturan pakai di kemasan obat un

In [8]:
print("Total noisy data generated:", augmented_cnt)
data.to_csv('./noisy/word-deletion.csv', index=False)

Total noisy data generated: 13


# COMBINED

In [19]:
prompt_template = f'''KALIMAT:

- lakukan 3 noise injection: typo, penghapusan kata, atau penyisipan kata-kata irrelevan yang membuat kalimat menjadi tidak nyambung
- lakukan noise injection pada maksimum 40% kata dari kalimat
- pastikan kalimat akhir memiliki masing-masing macam noise injection
- jika ada beberapa kata yang sama, hanya ubah satu kata saja
- jangan tambahkan markdown dan tanda baca apapun
- gunakan huruf kecil
- output hanya berupa kalimat akhir yang telah diubah
'''

In [20]:
data = load_data()
data.head()

,No,answer,1-FR,2-GI,3-PI,4-DM,5-EDTRB,6-RE,Text_Clean,filtered_text,token,tokens_stemmed,Process_Data
0,401,"Dear Ressa, Sakit bagian dada dapat disebabkan...",1,0,1,1,1,0,dear ressa sakit bagian dada dapat disebabkan ...,dear ressa sakit dada disebabkan cedera otot k...,"['dear', 'ressa', 'sakit', 'dada', 'disebabkan...","['dear', 'ressa', 'sakit', 'dada', 'sebab', 'c...",dear ressa sakit dada sebab cedera otot kram o...
1,402,"Alo, Terima kasih atas pertanyaannya. Perlu di...",1,0,1,1,1,0,alo terima kasih atas pertanyaannya perlu dipa...,alo terima kasih pertanyaannya alami pembengka...,"['alo', 'terima', 'kasih', 'pertanyaannya', 'a...","['alo', 'terima', 'kasih', 'tanya', 'alami', '...",alo terima kasih tanya alami bengkak benjol le...
2,403,Halo Virus HIV dapat ditularkan dengan adanya ...,1,0,1,1,1,0,halo virus hiv dapat ditularkan dengan adanya ...,halo virus hiv ditularkan kondisi bersamaan pe...,"['halo', 'virus', 'hiv', 'ditularkan', 'kondis...","['halo', 'virus', 'hiv', 'tular', 'kondisi', '...",halo virus hiv tular kondisi sama pindah cair ...
3,404,Hallo Imam Rasa gatal bisa timbul akibat ban...,1,0,1,1,1,0,hallo imam rasa gatal bisa timbul akibat banya...,hallo imam gatal timbul akibat faktor reaksi a...,"['hallo', 'imam', 'gatal', 'timbul', 'akibat',...","['hallo', 'imam', 'gatal', 'timbul', 'akibat',...",hallo imam gatal timbul akibat faktor reaksi a...
4,405,"Halo, Profertil berisi Clomiphene Citrate ya...",1,0,1,0,1,0,halo profertil berisi clomiphene citrate yang ...,halo profertil berisi clomiphene citrate obat ...,"['halo', 'profertil', 'berisi', 'clomiphene', ...","['halo', 'profertil', 'isi', 'clomiphene', 'ci...",halo profertil isi clomiphene citrate obat ran...


In [21]:
random.seed(42)
augmented_cnt = 0

for i in range(len(data)):
    n = random.uniform(0, 1)

    if n <= 0.1:
        augmented_cnt += 1
        text = data.iloc[i]['Text_Clean']

        print("Index:", i)
        print("Original:", text, '\n')

        augmented = generate_noise(build_prompt(text))
        data["Text_Clean"].values[i] = augmented

        print("Augmented:", augmented)
        print("=" * 50)

        time.sleep(2)

Index: 1
Original: alo terima kasih atas pertanyaannya perlu dipastikan kembali apakah yang anda alami berupa pembengkakan atau benjolan di leher karena kedua keluhan ini bisa disebabkan oleh kondisi yang berbeda beberapa kondisi medis dapat menyebabkan gejala yang anda alami sehingga untuk memastikannya diperlukan pemeriksaan dokter lebih lanjut beberapa penyebabnya misalnya reaksi alergi infeksi virus bakteri limfoma penyakit autoimun apabila gejala ini sudah berlangsung selama beberapa waktu silakan anda memeriksakan diri ke dokter umum terdekat dulu dokter perlu bertanya lebih lanjut mengenai gejala anda memeriksa anda dan bila perlu melakukan pemeriksaan lain seperti pemeriksaan darah untuk memastikan sebabnya dari pemeriksaan ini dokter akan memberikan penanganan lebih lanjut termasuk merujuk anda ke dokter spesialis bila diperlukan untuk sementara silakan melakukan anjuran berikut untuk mengurangi gejala anda mengkonsumsi parasetamol sesuai dengan aturan pakai di kemasan obat un

In [22]:
print("Total noisy data generated:", augmented_cnt)

Total noisy data generated: 13


In [23]:
data.to_csv('./noisy/combined-noise.csv', index=False)